# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data)
n_features = len(student_data.ix[:, student_data.columns != 'passed'].transpose())
n_passed = len(student_data[student_data['passed']=='yes'])
n_failed = len(student_data[student_data['passed']=='no'])
grad_rate = float(n_passed)/float(n_students)*100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all,y_all,test_size=num_test,random_state=0)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [7]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.svm import SVC
clf = SVC()
# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.013
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [8]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.007
F1 score for training set: 0.869198312236


In [9]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.758620689655


In [10]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))
# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

# Split training data into samples of sample_size
def split_training_samples(X_train, y_train, sample_size):
    # Split training sample into a 200 sample size
    train = pd.concat([X_train, y_train], axis=1)
    train = train.sample(n=sample_size)
    X_train_sample = train.ix[:, train.columns != 'passed']
    y_train_sample = train['passed']
    return X_train_sample, y_train_sample

X_train200, y_train200 = split_training_samples(X_train, y_train, 200)
X_train100, y_train100 = split_training_samples(X_train, y_train, 100)

# Train and get the F1 score for training and test.
train_predict(clf, X_train200, y_train200, X_test, y_test)
train_predict(clf, X_train100, y_train100, X_test, y_test)

------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.005
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.890365448505
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.791666666667
------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.002
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.872093023256
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.766233766234


In [12]:
# TODO: Train and predict using two other models

# I'm going to run the three models in order to compare the results together

# Function to save training time, prediction time, f1 score for training and test samples and sample size.
def model_performance(clf, X_train, y_train, X_test, y_test, table):
    # Sample Size
    sample_size = len(X_train)
    # Training
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    train_time = end - start
    f1_training = f1_score(y_train.values, clf.predict(X_train), pos_label='yes')
    # Test
    start = time.time()
    y_pred = clf.predict(X_test)
    end = time.time()
    test_time = end - start
    f1_test = f1_score(y_test.values, clf.predict(X_test), pos_label='yes')
    # Return the performance table with the new information  
    table.loc[len(table)+1]=[sample_size, train_time, f1_training, test_time, f1_test, clf.__class__.__name__]
    return table

# Get samples ready for all the models.
X_train200, y_train200 = split_training_samples(X_train, y_train, 200)
X_train100, y_train100 = split_training_samples(X_train, y_train, 100)

# Set up the DataFrame with the performance data
columns = ['sample_size','train_time','f1_training','test_time','f1_test','model']
table = pd.DataFrame(columns=columns)

# Support Vector Machines
clf = SVC()
table = model_performance(clf, X_train, y_train, X_test, y_test, table)
table = model_performance(clf, X_train200, y_train200, X_test, y_test, table)
table = model_performance(clf, X_train100, y_train100, X_test, y_test, table)

# Decision Tree Classifier
from sklearn import tree
clf = tree.DecisionTreeClassifier()
table = model_performance(clf, X_train, y_train, X_test, y_test, table)
table = model_performance(clf, X_train200, y_train200, X_test, y_test, table)
table = model_performance(clf, X_train100, y_train100, X_test, y_test, table)

# Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
table = model_performance(clf, X_train, y_train, X_test, y_test, table)
table = model_performance(clf, X_train200, y_train200, X_test, y_test, table)
table = model_performance(clf, X_train100, y_train100, X_test, y_test, table)

print table[table['model']=='SVC']
print table[table['model']=='DecisionTreeClassifier']
print table[table['model']=='GaussianNB']

#train_predict(clf, X_train, y_train, X_test, y_test)
#train_predict(clf, X_train200, y_train200, X_test, y_test)
#train_predict(clf, X_train100, y_train100, X_test, y_test)

   sample_size  train_time  f1_training  test_time   f1_test model
1          300    0.007171     0.869198   0.001912  0.758621   SVC
2          200    0.005202     0.873786   0.001408  0.783784   SVC
3          100    0.001315     0.860927   0.000802  0.786667   SVC
   sample_size  train_time  f1_training  test_time   f1_test  \
4          300    0.003873            1   0.000234  0.717949   
5          200    0.001344            1   0.000162  0.750000   
6          100    0.001019            1   0.000188  0.650794   

                    model  
4  DecisionTreeClassifier  
5  DecisionTreeClassifier  
6  DecisionTreeClassifier  
   sample_size  train_time  f1_training  test_time   f1_test       model
7          300    0.001139     0.808824   0.000297  0.750000  GaussianNB
8          200    0.001175     0.755020   0.000333  0.721311  GaussianNB
9          100    0.000880     0.414634   0.000337  0.423529  GaussianNB


In [13]:
# Graph F1 test scores for several training sample sizes
# Used this block to check how did the F1 score changed through different training samples sizes for each model.
import matplotlib.pyplot as pl
X_train, X_test, y_train, y_test = cross_validation.train_test_split(preprocess_features(student_data.ix[:, student_data.columns != 'passed'])
                                                                     ,student_data['passed'],test_size=num_test,random_state=0)

clf = SVC()
# Setting up the DataFrame to plot latter
columns = ['sample_size','train_time','f1_training','test_time','f1_test','model']
performance = pd.DataFrame(columns=columns)
# Run for every sample size from 300 to 0 by decreasing the sample size by 25
sample_size = 300
while sample_size != 0:
    X_train_it, y_train_it = split_training_samples(X_train, y_train, sample_size) 
    performance = model_performance(clf, X_train_it, y_train_it, X_test, y_test, performance)
    sample_size += -25

print performance
pl.figure()
pl.title('Training Sample Size vs Test F1 Score')
pl.plot(performance['sample_size'], performance['f1_test'], lw=2, label = 'F1 Score')
pl.legend()
pl.xlabel('Sample Size')
pl.ylabel('F1 Score')
pl.show()


    sample_size  train_time  f1_training  test_time   f1_test model
1           300    0.009662     0.869198   0.002260  0.758621   SVC
2           275    0.006137     0.873239   0.001490  0.767123   SVC
3           250    0.006924     0.874074   0.001613  0.778523   SVC
4           225    0.004189     0.868571   0.001251  0.767123   SVC
5           200    0.003526     0.858006   0.001132  0.766234   SVC
6           175    0.002640     0.885609   0.001052  0.768212   SVC
7           150    0.002087     0.881057   0.000939  0.789116   SVC
8           125    0.001673     0.850000   0.000853  0.778523   SVC
9           100    0.001300     0.817073   0.000741  0.776316   SVC
10           75    0.000895     0.844828   0.000613  0.766234   SVC
11           50    0.000597     0.829268   0.000465  0.771242   SVC
12           25    0.000445     0.871795   0.000349  0.760000   SVC


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [17]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn import grid_search
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

X_train, X_test, y_train, y_test = cross_validation.train_test_split(preprocess_features(student_data.ix[:, student_data.columns != 'passed'])
                                                                     ,student_data['passed'],test_size=num_test,random_state=0)

# I choose SVM as the best model.
# Setting up the parameters to use on grid search
parameters = {'C':(0.001,0.01,1,5,10,50,100,1000)}

clf = SVC()
f1_scorer = make_scorer(f1_score, pos_label="yes")
reg = grid_search.GridSearchCV(clf, parameters, scoring=f1_scorer)
reg.fit(X_train,y_train)
print "Best Estimator:"
print reg.best_estimator_
print "F1 score for test sample:"
predict_labels(reg, X_test, y_test)

Best Estimator:
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
F1 score for test sample:
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.002


0.75862068965517238